In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Dropout,Add,LayerNormalization,MultiHeadAttention,Embedding,Attention
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import random
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from datasets import load_dataset
dataset = load_dataset('dair-ai/emotion')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
tokenizer = Tokenizer()

In [ ]:
lens = []
for i in dataset['train']['text']:
    lens.append(len(i.split(' ')))
print(lens)

In [ ]:
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['validation']

tokenizer.fit_on_texts(train_dataset['text'])

train_texts = pad_sequences(tokenizer.texts_to_sequences(train_dataset['text']),maxlen=50,padding='post',truncating='post')
train_labels = train_dataset['label']
test_texts = pad_sequences(tokenizer.texts_to_sequences(test_dataset['text']),maxlen=50,padding='post',truncating='post')
test_labels = test_dataset['label']
validation_texts = pad_sequences(tokenizer.texts_to_sequences(validation_dataset['text']),maxlen=50,padding='post',truncating='post')
validation_labels = validation_dataset['label']


In [ ]:
def transformer_block(x,n_heads,key_dim):

    attn = MultiHeadAttention(num_heads=n_heads,key_dim=key_dim)(x,x)
    add = Add()([attn,x])
    normalization = LayerNormalization()(add)

    fc = Dense(key_dim,activation='relu')(normalization)
    fc = Dense(key_dim,activation='relu')(fc)
    x = Add()([normalization,fc])
    x = LayerNormalization()(x)

    return x

def build_model():

    text_input = keras.layers.Input(shape=(50,1))

    main = keras.layers.Flatten()(text_input)
    main = Embedding(len(tokenizer.word_index),32)(main)

    main = Attention()([main,main])
    main = transformer_block(main,n_heads=8,key_dim=32)

    main = keras.layers.GlobalAveragePooling1D(data_format='channels_first')(main)

    main = Dense(64,activation='relu')(main)
    main = Dropout(0.2)(main)
    main = Dense(32,activation='relu')(main)
    main = Dropout(0.2)(main)
    main = Dense(32,activation='relu')(main)

    output = Dense(6,activation='softmax')(main)

    print('v2')

    return keras.models.Model(inputs=text_input,outputs=output)

m=build_model()
m.summary()

#print(m.predict([np.random.rand(1,20),np.random.rand(1,200)]))

v2


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 50, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_6 (Flatten) │ (None, 50)        │          0 │ input_layer_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, 50, 32)    │    486,784 │ flatten_6[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_6         │ (None, 50, 32)    │          0 │ embedding_6[0][0… │
│ (Attention)         │                   │            │ embedding_6[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 50, 32)    │     33,568 │ attention_6[0][0… │
│ (MultiHeadAttentio… │                   │            │ attention_6[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_16 (Add)        │ (None, 50, 32)    │          0 │ multi_head_atten… │
│                     │                   │            │ attention_6[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 50, 32)    │         64 │ add_16[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_40 (Dense)    │ (None, 50, 32)    │      1,056 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_41 (Dense)    │ (None, 50, 32)    │      1,056 │ dense_40[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_17 (Add)        │ (None, 50, 32)    │          0 │ layer_normalizat… │
│                     │                   │            │ dense_41[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 50, 32)    │         64 │ add_17[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 50)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_42 (Dense)    │ (None, 64)        │      3,264 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_21          │ (None, 64)        │          0 │ dense_42[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 32)        │      2,080 │ dropout_21[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_22          │ (None, 32)        │          0 │ dense_43[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_44 (Dense)    │ (None, 32)        │      1,056 │ dropout_22[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_45 (Dense)    │ (None, 6)         │        198 │ dense_44[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 529,190 (2.02 MB)

 Trainable params: 529,190 (2.02 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print(np.unique(train_labels,return_counts=True))

(array([0, 1, 2, 3, 4, 5]), array([4666, 5362, 1304, 2159, 1937,  572]))


In [ ]:
model = build_model()

model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=0.00001),metrics=['acc'])

bs = 4

train_labels = np.expand_dims(train_labels,axis=1)
validation_labels = np.expand_dims(validation_labels,axis=1)

train_inp = tf.data.Dataset.from_tensor_slices((np.expand_dims(train_texts,axis=1),train_labels))
val_inp =  tf.data.Dataset.from_tensor_slices((np.expand_dims(validation_texts,axis=1),validation_labels))

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,start_from_epoch=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=5,start_from_epoch=10)

class_weight = dict(enumerate(compute_class_weight(class_weight='balanced',classes=np.unique(train_labels),y=train_labels.flatten())))

model.fit(train_inp,validation_data=val_inp,batch_size=bs,epochs=100,class_weight=class_weight,callbacks=[early_stop,reduce_lr])

v2
Epoch 1/100
16000/16000 ━━━━━━━━━━━━━━━━━━━━ 66s 4ms/step - acc: 0.1270 - loss: 1.8142 - val_acc: 0.1060 - val_loss: 1.7907 - learning_rate: 1.0000e-05
Epoch 2/100
16000/16000 ━━━━━━━━━━━━━━━━━━━━ 57s 4ms/step - acc: 0.1680 - loss: 1.8144 - val_acc: 0.1060 - val_loss: 1.7898 - learning_rate: 1.0000e-05
Epoch 3/100
16000/16000 ━━━━━━━━━━━━━━━━━━━━ 58s 4ms/step - acc: 0.2223 - loss: 1.8145 - val_acc: 0.1060 - val_loss: 1.7895 - learning_rate: 1.0000e-05
Epoch 4/100
16000/16000 ━━━━━━━━━━━━━━━━━━━━ 56s 4ms/step - acc: 0.2767 - loss: 1.8145 - val_acc: 0.1060 - val_loss: 1.7895 - learning_rate: 1.0000e-05
Epoch 5/100
16000/16000 ━━━━━━━━━━━━━━━━━━━━ 57s 4ms/step - acc: 0.2014 - loss: 1.8143 - val_acc: 0.1345 - val_loss: 1.7877 - learning_rate: 1.0000e-05
Epoch 6/100
16000/16000 ━━━━━━━━━━━━━━━━━━━━ 57s 4ms/step - acc: 0.2413 - loss: 1.8048 - val_acc: 0.3510 - val_loss: 1.7424 - learning_rate: 1.0000e-05
Epoch 7/100
16000/16000 ━━━━━━━━━━━━━━━━━━━━ 82s 4ms/step - acc: 0.3245 - loss: 1.733

In [ ]:
i = random.randint(0,len(test_dataset['text']))
text = test_texts[i]
text = np.reshape(text,(1,50,1))
label = test_labels[i]
print(classes[label])
print(test_dataset['text'][i])

classes = ['joy','sadness','anger','fear','love','surprise']
classes = dict(enumerate(classes))

#text = pad_sequences(tokenizer.texts_to_sequences(test),maxlen=50,padding='post').reshape(1,50,1)

probs = model.predict(text)[0]
print(probs)
prediction = classes[np.argmax(probs)]

print(prediction)

love
i just feel you so so don t be afraid i should hurt even more and pray again so i can find you again the more time passes the more it hurts i need you go back in time just one time forgive my sins if only i could turn things back this pain would be so so sloth
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[2.7202802e-02 1.8404889e-12 9.8010058e-18 4.8263860e-10 9.7279716e-01
 1.0040498e-20]
love


In [ ]:
test_x = tf.data.Dataset.from_tensor_slices((np.expand_dims(test_texts,axis=1),np.expand_dims(test_labels,axis=1)))
model.evaluate(test_x)

2000/2000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - acc: 0.7658 - loss: 0.8293


[0.9176890850067139, 0.7475000023841858]

The runtime disconnected due to free tier limits, but in the above evaluation, you can see it in the format [loss,accuracy]

final metrics (approsimate):
train accuracy : 90%
validation accuracy : 74%
test accuracy : 74%